In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv(r'C:\Users\asus\Desktop\DA_projects\karpov\band5\task1\data.csv',sep=',', encoding= 'ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [19]:
'''2) Проверьте, встречаются ли в данных повторяющиеся наблюдения,
и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.'''
df = df.drop_duplicates()

In [25]:
'''3) ⭐️Задание со звёздочкой!⭐️
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных.
Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?'''
df[df['InvoiceNo'].apply(lambda x:x.startswith('C'))].shape[0]

9251

In [29]:
'''4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. 
В качестве ответа укажите число оставшихся строк.'''
df = df[df['Quantity']>0]
df.shape[0]

526054

In [70]:
'''5) 💀 Сложное задание! 💀
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany).
Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 
Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. 
Для каждого заказа в данных может встречаться более 1 строки.'''
german = df[df['Country']=='Germany'].groupby('CustomerID',as_index=False).agg({'InvoiceNo':'nunique'})

In [62]:
german = df[df['Country']=='Germany'].groupby('CustomerID',as_index=False).agg({'InvoiceNo':'nunique'})

In [69]:
german_top = german[german['InvoiceNo']>german['InvoiceNo'].quantile(0.8)]['CustomerID']
german_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

In [72]:
'''6) ⭐️Задание со звёздочкой!⭐️
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, 
отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. 
Результирующий датафрейм запишите в top_retail_germany.'''
top_retail_germany = pd.merge(df,german_top,how='inner',on='CustomerID')
top_retail_germany.head(6)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
5,536527,22243,5 HOOK HANGER RED MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany


In [92]:
'''7) Сгруппируйте top_retail_germany по коду товара (StockCode).
Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.
'''
top_retail_germany[top_retail_germany['StockCode']!='POST']['StockCode'].value_counts().idxmax()

'22326'

In [94]:
'''8) Вернемся к анализу полного датасета retail. 
Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.'''
df['Revenue'] = df['Quantity']*df['UnitPrice']

In [111]:
'''9) Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.
В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)
'''
top_invoince = df.groupby('InvoiceNo',as_index=False).agg({'Revenue':'sum'}) \
    .sort_values('Revenue',ascending=False)['InvoiceNo'].head(5)
print(top_invoince.str.cat(sep=', '))

581483, 541431, 574941, 576365, 556444
